In [ ]:
%env OPENAI_API_KEY=
%env COHERE_API_KEY=

# Why LangChain?

### There are many reasons, let's look at one

### Let's say we want OpenAI GPT models to write us a joke

In [2]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.Completion.create(
  model="text-davinci-003",
  prompt="Tell me a joke on procrastination",
  max_tokens=35,
  temperature=0.7
)

In [3]:
response

<OpenAIObject text_completion id=cmpl-7TFFyHU8Q2UTm8uasE2vieHSQfhfQ at 0x1130d7170> JSON: {
  "id": "cmpl-7TFFyHU8Q2UTm8uasE2vieHSQfhfQ",
  "object": "text_completion",
  "created": 1687204514,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\n\nQ: Why did the procrastinator eat breakfast in bed?\nA: Because he wanted to procrastinate on getting out of it.",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 8,
    "completion_tokens": 31,
    "total_tokens": 39
  }
}

In [4]:
response.choices[0].text

'\n\nQ: Why did the procrastinator eat breakfast in bed?\nA: Because he wanted to procrastinate on getting out of it.'

In [5]:
import cohere
co = cohere.Client(os.getenv("COHERE_API_KEY"))

response_cohere = co.generate(
  prompt='Tell me a joke on procrastination',
)

In [6]:
response_cohere

,prompt,text
0,Tell me a joke on procrastination,Why did the student put off writing his essay? Because he was too lazy to start!


In [7]:
print(response_cohere[0])


Why did the student put off writing his essay?
Because he was too lazy to start!


## How about if you would like to call LLMs from other providers/platforms

- AI21
- Aleph Alpha
- Hugging Face Hub
- GPT4All
- ...


## Let's try again with LangChain

In [8]:
from langchain.llms import OpenAI

llm = OpenAI()

In [9]:
llm.predict("Tell me a joke on procrastination")

"\n\nQ: Why did the procrastinator cross the road?\nA: He'll do it tomorrow!"

In [10]:
from langchain.llms import Cohere

llm = Cohere()

In [11]:
llm.predict("Tell me a joke on procrastination")

'\nWhy did the student put off studying for his exam?\nBecause he was too lazy to start.'

#### Most of the heavylifting is done for you, just a swap of provider/platform, and you are all set

#### What if you would like to swap the joke topic

"Tell me a joke on {user_topic}"

```
joke_template ="Tell me a joke on {user_topic}"
```

Use the Prompt Templates available in LangChain and supply the joke template created

```
joke_template ="Tell me a joke on {user_topic}"  

PromptTemplate(template=joke_template)
```

Also specify the input variables, in our case - user_topic

```
PromptTemplate(template=joke_template, 
                input_variables=["user_topic"])
```

In [12]:
from langchain import PromptTemplate

joke_template ="Tell me a joke on {user_topic}"

prompt = PromptTemplate(template=joke_template, input_variables=["user_topic"])

In [13]:
prompt.format(user_topic="procrastination")

'Tell me a joke on procrastination'

In [14]:
final_prompt = prompt.format(user_topic="procrastination")

In [15]:
llm.predict(final_prompt)

'\nWhy did the student put off his homework?\nHe wanted to make it a perfect ten.'

#### What if we want to get the user_topic for the joke, and also have the flexibility to use different LLM providers?

For example:  

prompt = PromptTemplate  
user_topic = "taxes"  

llm = "Cohere"

Old:  
```
prompt = PromptTemplate(template=joke_template, 
                        input_variables=["user_topic"])  
                        
final_prompt = prompt.format(user_topic="taxes")  


llm.predict(final_prompt)
```

#### For that we chain LLM and prompt together

New:  
```
prompt = PromptTemplate(template=joke_template, 
                        input_variables=["user_topic"])  
llm= Cohere()
```

```
LLMChain(prompt=prompt, llm=Cohere())
```

In [16]:
from langchain.chains import LLMChain

llm_joke_chain = LLMChain(prompt=prompt, llm=Cohere())

In [17]:
llm_joke_chain.predict(user_topic="taxes")

'\nWhy did the taxpayer cross the road?\nTo get to the other tax office.'

#### How about a list of user_topics

In [19]:
input_list = [
    {"user_topic": "procrastination"},
    {"user_topic": "taxes"},
    {"user_topic": "Canada"}
]

llm_joke_chain.apply(input_list)

[{'text': '\nWhy did the student put off studying for the exam?\nBecause he knew the teach would give him extra time to study.'},
 {'text': '\nWhy did the taxpayer cross the road?\nBecause the other side had a lower tax rate.'},
 {'text': '\nWhy did the Canadian cross the road?\nTo get to the Tim Hortons on the other side!'}]

#### What if you would like to first get a joke, and then write a NY Times Article about it?

#### Use Sequential Chain

In [20]:
llm = OpenAI(temperature=.7)
template = """You are a NY Times Editor. Given the joke by an AI, it is your job to write a NY Times article on the joke.

Joke: {joke}
Article: This is the article about the joke above:"""
prompt_template = PromptTemplate(input_variables=["joke"], template=template)
ny_times_article_chain = LLMChain(llm=llm, prompt=prompt_template)

In [21]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[llm_joke_chain, ny_times_article_chain], verbose=True)

In [22]:
article = overall_chain.run("Canada")



> Entering new  chain...

Why did the Canadian cross the road?
To get to the Tim Hortons on the other side!


Canadians are known for their love of timbits, poutine, and maple syrup, but the latest trend for Canadians is crossing the road.

That's right, Canadians are crossing the road to get to the one place they love most, Tim Hortons.

The "Tim Hortons Crossing" phenomenon has been sweeping the nation. People are leaving their homes and braving traffic to get to the beloved coffee chain.

So why are Canadians taking this risk? It's all for a good cup of coffee and a donut or two. Many people say they just can't resist the warm atmosphere and friendly staff at Tim Hortons.

It's clear that Tim Hortons has become a national institution for Canadians. It's not just a place to get coffee, but a place to meet up with friends or just relax and enjoy the moment.

So, the next time you see a Canadian crossing the road, you'll know why.

> Finished chain.


#### How about we ask a question from the provided story?

We can use a Question Answering chain

In [23]:
from langchain.chains.question_answering import load_qa_chain

qa_chain = load_qa_chain(llm=llm)


In [24]:
from langchain.chains import AnalyzeDocumentChain
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

In [25]:
qa_document_chain.run( input_document=article, 
                          question="what is mentioned about Canadians")

' Canadians are known for their love of timbits, poutine, and maple syrup, and they have recently been crossing roads to get to Tim Hortons for a good cup of coffee and donut or two. They find the atmosphere and staff at Tim Hortons welcoming, and it has become a national institution.'

#### What if we want to ask question from a large text body eg. books, policies, code bases etc

We use a path similar to the diagram below:
![diagram](https://langchain-langchain.vercel.app/assets/images/data_connection-c42d68c3d092b85f50d08d4cc171fc25.jpg)

In [26]:
#imports
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import CohereEmbeddings



In [27]:
loader = TextLoader('state_of_the_union.txt', encoding='utf8')

In [28]:
index = VectorstoreIndexCreator(embedding=CohereEmbeddings()).from_loaders([loader])

In [29]:
query = "What did the president say about Ketanji Brown Jackson"
index.query(query)

" The president said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence."

### There is a tremendous opportunity to combine the hundreds of loaders, embedding models, vectorstores, and retrievers. Build something fun and share with the community!


### What if we want our application to remember our conversation

#### We can add "memory" 

In [30]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

In [31]:
from langchain.chains import ConversationChain


llm = OpenAI(temperature=0)
memory_chain = ConversationChain(llm=llm, memory=ConversationBufferMemory())

In [32]:
memory_chain.predict(input="Hi there! My name is Misbah, what's your name")

" Hi Misbah! My name is AI-1. It's nice to meet you."

In [33]:
memory_chain.predict(input="Can you remind me what's my name?")

' Sure, your name is Misbah.'